## Import dependencies

In [19]:
import random
import asyncio
import json
import time

## Code snippets

In [20]:
def randomize(min_val=1, max_val=10):
    return random.randint(min_val, max_val)

async def jitter(min_delay=1, max_delay=10):
    delay = randomize(min_delay, max_delay)
    await asyncio.sleep(delay/1000)

def split_string(text):
    result = []
    index = 0
    while index < len(text):
        token_length = randomize()
        token = text[index:index+token_length]
        result.append(token)
        index += token_length
    return result

def generate_stream_chunks(text):
  tokens = split_string(text)
  data = [
    {
      'id': str(index),
      'system_fingerprint': f'{int(time.time())}-{index}',
      'content': item,
    } for index, item in enumerate(tokens)
  ]
  return [f'{json.dumps(item)}\n' for item in data]

async def mock_stream_answer(text, min_delay=0, max_delay=300):
  chunks = generate_stream_chunks(text)

  for chunk in chunks:
    await jitter(min_delay, max_delay)
    yield json.loads(chunk)


## Test

In [ ]:
async def main():
    def generate_lorem_ipsum(length):
        lorem_ipsum = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed non risus. Suspendisse lectus tortor, dignissim sit amet, adipiscing nec, ultricies sed, dolor. Cras elementum ultrices diam. Maecenas ligula massa, varius a, semper congue, euismod non, mi."
        result = ""
        while len(result) < length:
            result += lorem_ipsum
        return result[:length]

    text = generate_lorem_ipsum(300)

    async for chunk in mock_stream_answer(text):
        print(chunk)

await main()